In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import matplotlib.animation as animation
from IPython.display import HTML
from IPython.display import Image

#Code to always set working directory one level up so we can access /data easily
import os
if "NOTEBOOK_DIR" not in globals():
    NOTEBOOK_DIR = os.getcwd()
    print('Notebook Directory Set:', os.getcwd())

os.chdir(os.path.join(NOTEBOOK_DIR, ".."))
print("Current working directory:", os.getcwd())

Notebook Directory Set: /shared_folder/eas_508_project/notebooks
Current working directory: /shared_folder/eas_508_project


In [ ]:
pbp_path = 'data/raw/pbp_data/'

pbp_name = 'pbp_1999.csv'

df = pd.read_csv(pbp_path + pbp_name)

/tmp/ipykernel_43425/1153630516.py:5: DtypeWarning: Columns (36,37,179,180,189,190,197,198,203,204,205,206,213,214,218,219,220,222,224,226,248,249,253,254,255,260,262,263,266,267,268,269,283,284) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(pbp_path + pbp_name)


In [90]:
#From error
columns_to_fix = list(df.iloc[:, [36,37,179,180,189,190,197,198,203,204,205,206,213,214,218,219,220,222,224,226,248,249,253,254,255,260,262,263,266,267,268,269,283,284]].columns)

df.iloc[:, [36,37,179,180,189,190,197,198,203,204,205,206,213,214,218,219,220,222,224,226,248,249,253,254,255,260,262,263,266,267,268,269,283,284]].notna().sum()

pass_length                             344
pass_location                           351
lateral_receiver_player_id                4
lateral_receiver_player_name              4
lateral_interception_player_id            5
lateral_interception_player_name          5
lateral_kickoff_returner_player_id       10
lateral_kickoff_returner_player_name     10
own_kickoff_recovery_player_id           16
own_kickoff_recovery_player_name         16
blocked_player_id                        42
blocked_player_name                      38
qb_hit_2_player_id                      109
qb_hit_2_player_name                    109
forced_fumble_player_2_team               2
forced_fumble_player_2_player_id          2
forced_fumble_player_2_player_name        2
solo_tackle_2_team                      128
solo_tackle_2_player_id                 133
solo_tackle_2_player_name               128
pass_defense_2_player_id                 44
pass_defense_2_player_name               44
fumbled_2_player_id             

In [96]:
type_dict = {}
for item in columns_to_fix:
    type_dict[item] = df[f'{item}'].value_counts().to_dict()

type_df = pd.DataFrame(type_dict)

max_len = max(len(v) for v in type_dict.values())

# Build a DataFrame where each column lists its unique values
df_uniques = pd.DataFrame({
    col: list(subdict.keys()) + [None] * (max_len - len(subdict))
    for col, subdict in type_dict.items()
})

df_uniques.to_csv('data/interim/dtype_fix.csv', index=False)

In [ ]:
#Looking through the data we see each item can be done as a string, 2 can be considered categorical but for our purposes is not needed. Thus we can use:

dtypes = {item : "string" for item in columns_to_fix}

In [104]:
df = pd.read_csv(pbp_path + pbp_name, dtype=dtypes)

df

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,out_of_bounds,home_opening_kickoff,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,xpass,pass_oe
0,35,1999_01_ARI_PHI,1999091200,PHI,ARI,REG,1,PHI,home,ARI,...,0,1,0.126818,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,60,1999_01_ARI_PHI,1999091200,PHI,ARI,REG,1,PHI,home,ARI,...,0,1,-0.561568,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,82,1999_01_ARI_PHI,1999091200,PHI,ARI,REG,1,PHI,home,ARI,...,0,1,-0.641717,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,103,1999_01_ARI_PHI,1999091200,PHI,ARI,REG,1,PHI,home,ARI,...,0,1,-0.723302,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,126,1999_01_ARI_PHI,1999091200,PHI,ARI,REG,1,PHI,home,ARI,...,0,1,0.212661,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46131,4034,1999_21_STL_TEN,2000013000,TEN,LA,POST,21,TEN,home,LA,...,0,0,-0.193368,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46132,4056,1999_21_STL_TEN,2000013000,TEN,LA,POST,21,TEN,home,LA,...,0,0,-0.318140,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46133,4080,1999_21_STL_TEN,2000013000,TEN,LA,POST,21,TEN,home,LA,...,0,0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46134,4097,1999_21_STL_TEN,2000013000,TEN,LA,POST,21,TEN,home,LA,...,0,0,-2.277798,NaN,NaN,NaN,NaN,NaN,NaN,NaN
